In [1]:
import litellm
from litellm import completion
import numpy as np
import pandas as pd
litellm.drop_params = True

In [2]:
def call_llm(prompt, **kwargs):
    if "temperature" not in kwargs:
        kwargs["temperature"] = 1.2
    response = completion( # OpenAI-compatible completion function
        model="ollama/llama3.1", 
        messages=[{ "content": prompt,"role": "user"}], 
        api_base="http://localhost:11434",
        **kwargs
    )
    choice = response.choices[0]
    if choice.finish_reason == "tool_calls":
        tool_call_data = choice.message.tool_calls[0]
        tool_call_str = f"{tool_call_data.function.name}(**{tool_call_data.function.arguments})"
        return eval(tool_call_str)
    else:
        return choice.message.content

In [3]:
reviews = [
    "I loved this movie!",
    "I hated this movie!",
    "I watched this movie.",
]

In [4]:
zero_shot_sentiment_template = """
Classify the sentiment of the following text as positive, negative, or neutral. \
Only return the classification. 
Text: {text}
Answer: \
"""
for review in reviews:
    print(review)
    print(call_llm(zero_shot_sentiment_template.format(text=review)))

I loved this movie!
Positive
I hated this movie!
Negative
I watched this movie.
Neutral


In [5]:
few_shot_sentiment_template = """
Classify the sentiment of the following text as positive, negative, or neutral. \
Only return the classification. \
Examples:

Text: The food was delicious
Answer: positive

Text: I did not enjoy the scenery
Answer: negative

Text: Go straight then turn left at the stop sign
Answer: neutral

Text: It smelled so bad
Answer: negative

Text: I had a great time
Answer: positive

Text: {text}
Answer: \
"""
for review in reviews:
    print(review)
    print(call_llm(few_shot_sentiment_template.format(text=review)))

I loved this movie!
positive
I hated this movie!
negative
I watched this movie.
neutral


In [5]:
for _ in range(3):
    print(call_llm("What is (1+3)^2 - 2? Let's think step by step.", temperature=1.99))
    print("=" * 50)

To calculate this expression, we need to follow the order of operations: parentheses, exponents, multiplication and division (from left to right), and addition and subtraction (from left to right).

Here are the steps:

1. First, let's solve the expression inside the parentheses:
(1 + 3) = 4
So, the original expression becomes:
(4)^2 - 2

2. Now, we need to calculate the square of 4:
(4)^2 = 16

3. Finally, subtract 2 from 16:
16 - 2 = 14

Therefore, (1+3)^2 - 2 = 14.
To solve this expression, we'll follow the order of operations, often remembered by the acronym PEMDAS:

1. Parentheses: Evaluate expressions inside parentheses first.
2. Exponents: Next, calculate any exponents (such as squaring).
3. Multiplication and Division: Perform these operations from left to right.
4. Addition and Subtraction: Finally, perform these operations from left to right.

### Step 1: Evaluate the Expression Inside Parentheses
First, let's solve the expression inside parentheses, which is \(1 + 3\).

\(1 

In [6]:
def get_current_weather(city):
    weather = np.random.choice(["sunny", "cloudy", "rainy", "snowy"])
    return f"The weather in {city} is {weather}."

In [7]:
tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ],

In [8]:
for _ in range(5):
    print(call_llm("What is the weather in Paris?", tools=tools))

The weather in Paris is sunny.
The weather in Paris is rainy.
The weather in Paris is rainy.
The weather in Paris is sunny.
The weather in Paris is cloudy.


In [21]:
df = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David"],
    "age": [25, 30, 35, 40],
    "favorite_color": ["blue", "red", "blue", "green"],
})

# question = "Create a bar plot indicating the frequency of favorite colors."
question = "Create a box and whisker plot of age using plotly."

df_sample = df.sample(3).to_markdown()
df_describe = df.describe().to_markdown()
df_info = str(df.info())

prompt = f"""\
Below is some information about a pandas dataframe df. Here is the output of df.sample().to_markdown():
{df_sample}
Here is the output of df.describe().to_markdown():
{df_describe}
Here is the output of str(df.info()):
{df_info}
Please write python code to answer the following question using data in the dataframe. \
The code must start with ```python\n and end with ```. \
The python will be executed with eval(), assume the data is already loaded in a dataframe df.
Question:
{question}
Python code: """

output = call_llm(prompt)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            4 non-null      object
 1   age             4 non-null      int64 
 2   favorite_color  4 non-null      object
dtypes: int64(1), object(2)
memory usage: 228.0+ bytes


In [22]:
def extract_python(output):
    start = "```python\n"
    end = "```"
    start_idx = output.find(start) + len(start)
    end_idx = output.find(end, start_idx)
    return output[start_idx:end_idx]

In [23]:
print(extract_python(output))

import plotly.express as px

fig = px.box(df, x="age")
fig.show()



In [24]:
exec(extract_python(output))

In [37]:
df

,name,age,favorite_color
0,Alice,25,blue
1,Bob,30,red
2,Charlie,35,blue
3,David,40,green
